In [52]:
import os
import platform
from IPython.display import clear_output
print(platform.platform())

def resolve_dir(Dir):
    if not os.path.exists(Dir):
        os.mkdir(Dir)

def reset_path(Dir):
    if not os.path.exists(Dir):
        os.mkdir(Dir)
    else:
        os.system('rm -f {}/*'.format( Dir))

macOS-13.2.1-arm64-arm-64bit


In [53]:
import tensorflow as tf
tf.random.set_seed(73)
TPU_INIT = False

if TPU_INIT:
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
        tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)
    
    except ValueError:
        raise BaseException('ERROR: Not connected to a TPU runtime!')
else:
    !nvidia-smi
;    
print("Tensorflow version " + tf.__version__)


zsh:1: command not found: nvidia-smi
Tensorflow version 2.13.0-rc2


In [54]:
#416*416*3 is the image size as it is large enough where here is enough pixel 
# clarity and small enough that our model can learn well/we can train it well

In [55]:
print('we have \n{} NonViolence videos \n{} Violence videos'.format(
              len(os.listdir('/Users/faizahkureshi/Desktop/GAIP Project/Image/NonViolence')), 
              len(os.listdir('/Users/faizahkureshi/Desktop/GAIP Project/Image/Violence'))))

we have 
183 NonViolence videos 
1890 Violence videos


In [56]:
import cv2
import os
import imageio
import imgaug.augmenters as iaa
import imgaug as ia

def aug_img(image):

    IMG_SIZE = 128
    ColorChannels = 3
    image = cv2.imread(image)
    resized_image = cv2.resize(image, (128, 128))
    


In [57]:
%%time
#tqdm is used here to visulaize the progress of dataset creation
from tqdm import tqdm

IMG_SIZE=128
ColorChannels=3

ImgDataDir = '/Users/faizahkureshi/Desktop/GAIP Project/Image'

X_original = []
y_original = []
CLASSES = ["NonViolence", "Violence"]

for category in os.listdir(ImgDataDir):
    category_path = os.path.join(ImgDataDir, category)
    if os.path.isdir(category_path):
        class_num = CLASSES.index(category)
        try:
            img_list = [image for image in os.listdir(category_path)[:350] if image != '.DS_Store']
            for i, image_filename in enumerate(tqdm(img_list)):
                image_path = os.path.join(category_path, image_filename)
                image = cv2.imread(image_path)
                rgb_img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                resized = cv2.resize(rgb_img, (IMG_SIZE, IMG_SIZE))
                X_original.append(resized)
                y_original.append(class_num)
        except OSError as e:
            print(f"Error processing directory: {category_path}. Skipping...")
            continue


100%|██████████| 350/350 [00:00<00:00, 771.84it/s]

CPU times: user 550 ms, sys: 58.8 ms, total: 609 ms
Wall time: 717 ms


In [58]:
len(y_original)

532

In [59]:
import numpy as np
IMG_SIZE=128
X_original = np.array(X_original).reshape(532,IMG_SIZE * IMG_SIZE * 3)
y_original = np.array(y_original)
len(X_original)

532

In [60]:
len(y_original)

532

In [61]:
from sklearn.model_selection import StratifiedShuffleSplit

stratified_sample = StratifiedShuffleSplit(n_splits=2, test_size=0.3, random_state=73)

for train_index, test_index in stratified_sample.split(X_original, y_original):
    X_train, X_test = X_original[train_index], X_original[test_index]
    y_train, y_test = y_original[train_index], y_original[test_index]

X_train_nn = X_train.reshape(-1, IMG_SIZE, IMG_SIZE, 3) / 255
X_test_nn = X_test.reshape(-1, IMG_SIZE, IMG_SIZE, 3) / 255

In [62]:
from tensorflow.keras.layers import Input, Dropout, Flatten, Dense
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import pickle
import matplotlib
matplotlib.use("Agg")

In [63]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [64]:
epochs = 50

from keras import regularizers
kernel_regularizer = regularizers.l2(0.0001)

from keras.applications.mobilenet_v2 import MobileNetV2

def load_layers():
    input_tensor = Input(shape=(IMG_SIZE, IMG_SIZE, ColorChannels))
    baseModel = MobileNetV2(pooling='avg',
                            include_top=False, 
                            input_tensor=input_tensor)
    
    headModel = baseModel.output   
    headModel = Dense(1, activation="sigmoid")(headModel)
    model = Model(inputs=baseModel.input, outputs=headModel)

    for layer in baseModel.layers:
        layer.trainable = False

    print("Compiling model...")
    model.compile(loss="binary_crossentropy",
                    optimizer='adam',
                    metrics=["accuracy"])

    return model

if TPU_INIT:
    with tpu_strategy.scope():
        model = load_layers()
else:
    model = load_layers()

model.summary()

Compiling model...
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 128, 128, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 64, 64, 32)           864       ['input_1[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 64, 64, 32)           128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 64, 64, 32)           0         ['bn_Co

In [65]:
from tensorflow.keras.callbacks import Callback, ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping, ReduceLROnPlateau

patience = 3

start_lr = 0.00001
min_lr = 0.00001
max_lr = 0.00005

batch_size = 4

if TPU_INIT:
    max_lr = max_lr * tpu_strategy.num_replicas_in_sync
    batch_size = batch_size * tpu_strategy.num_replicas_in_sync

rampup_epochs = 5
sustain_epochs = 0
exp_decay = .8

def lrfn(epoch):
    if epoch < rampup_epochs:
        return (max_lr - start_lr)/rampup_epochs * epoch + start_lr
    elif epoch < rampup_epochs + sustain_epochs:
        return max_lr
    else:
        return (max_lr - min_lr) * exp_decay**(epoch-rampup_epochs-sustain_epochs) + min_lr


class myCallback(Callback):
    def on_epoch_end(self, epoch, logs={}):
        if ((logs.get('accuracy')>=0.999)):
            print("\nLimits Reached cancelling training!")
            self.model.stop_training = True

In [66]:
end_callback = myCallback()

lr_callback = LearningRateScheduler(lambda epoch: lrfn(epoch), verbose=False)

early_stopping = EarlyStopping(patience = patience, monitor='val_loss',
                                 mode='min', restore_best_weights=True, 
                                 verbose = 1, min_delta = .00075)

PROJECT_DIR ='/Users/faizahkureshi/Desktop/GAIP Project/RiskDetection'

lr_plat = ReduceLROnPlateau(patience = 2, mode = 'min')

os.system('rm -rf ./logs/')

import datetime
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir = log_dir, write_graph=True, histogram_freq=1)

checkpoint_filepath = 'ModelWeights.h5'

model_checkpoints = ModelCheckpoint(filepath=checkpoint_filepath,
                                        save_weights_only=True,
                                        monitor='val_loss',
                                        mode='min',
                                        verbose = 1,
                                        save_best_only=True)


callbacks = [end_callback, lr_callback, model_checkpoints, tensorboard_callback, early_stopping, lr_plat]

if TPU_INIT:
    callbacks = [end_callback, lr_callback, model_checkpoints, early_stopping, lr_plat]

In [67]:
print('Training head...')
#model.load_weights('./Model_Weights.h5')

history = model.fit(X_train_nn ,y_train, epochs=epochs,
                        callbacks=callbacks,
                        validation_data = (X_test_nn, y_test),
                        batch_size=batch_size)

print('\nRestoring best Weights for MobileNetV2')
model.load_weights(checkpoint_filepath)

Training head...
Epoch 1/50
91/93 [============================>.] - ETA: 0s - loss: 1.4242 - accuracy: 0.4121
Epoch 1: val_loss improved from inf to 1.20365, saving model to ModelWeights.h5
93/93 [==============================] - 4s 36ms/step - loss: 1.4260 - accuracy: 0.4113 - val_loss: 1.2036 - val_accuracy: 0.4437 - lr: 1.0000e-05
Epoch 2/50
92/93 [============================>.] - ETA: 0s - loss: 1.1408 - accuracy: 0.4647
Epoch 2: val_loss improved from 1.20365 to 0.89645, saving model to ModelWeights.h5
93/93 [==============================] - 3s 30ms/step - loss: 1.1324 - accuracy: 0.4677 - val_loss: 0.8964 - val_accuracy: 0.4938 - lr: 1.8000e-05
Epoch 3/50
93/93 [==============================] - ETA: 0s - loss: 0.8558 - accuracy: 0.5645
Epoch 3: val_loss improved from 0.89645 to 0.69246, saving model to ModelWeights.h5
93/93 [==============================] - 3s 28ms/step - loss: 0.8558 - accuracy: 0.5645 - val_loss: 0.6925 - val_accuracy: 0.6125 - lr: 2.6000e-05
Epoch 4/50
9

In [68]:
%matplotlib inline

def print_graph(item, index, history):
    plt.figure()
    train_values = history.history[item][0:index]
    plt.plot(train_values)
    test_values = history.history['val_' + item][0:index]
    plt.plot(test_values)
    plt.legend(['training','validation'])
    plt.title('Training and validation '+ item)
    plt.xlabel('epoch')
    plt.show()
    plot = '{}.png'.format(item)
    plt.savefig(plot)


def get_best_epoch(test_loss, history):
    for key, item in enumerate(history.history.items()):
        (name, arr) = item
        if name == 'val_loss':
            for i in range(len(arr)):
                if round(test_loss, 2) == round(arr[i], 2):
                    return i
                
def model_summary(model, history):
    print('---'*30)
    test_loss, test_accuracy = model.evaluate(X_test_nn, y_test, verbose=0)

    if history:
        index = get_best_epoch(test_loss, history)
        print('Best Epochs: ', index)

        train_accuracy = history.history['accuracy'][index]
        train_loss = history.history['loss'][index]

        print('Accuracy on train:',train_accuracy,'\tLoss on train:',train_loss)
        print('Accuracy on test:',test_accuracy,'\tLoss on test:',test_loss)
        print_graph('loss', index, history)
        print_graph('accuracy', index, history)
        print('---'*30)   

In [69]:
# evaluate the network
print("Evaluating network...")
predictions = model.predict(X_test_nn)
preds = predictions > 0.5

Evaluating network...
5/5 [==============================] - 1s 74ms/step


In [70]:
args_model = "modelnewimg.h5"
model.save(args_model)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [1]:
pip install -U ultralytics

     |████████████████████████████████| 626 kB 5.0 MB/s eta 0:00:01
     |████████████████████████████████| 173 kB 23.0 MB/s eta 0:00:01
     |████████████████████████████████| 55.8 MB 249 kB/s  eta 0:00:01
     |████████████████████████████████| 1.4 MB 13.7 MB/s eta 0:00:01
     |████████████████████████████████| 5.7 MB 10.4 MB/s eta 0:00:01
     |████████████████████████████████| 536 kB 10.3 MB/s eta 0:00:01
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
